In [1]:
#@title RWKV World 微调 （Alpha）
#@markdown RWKV World 微调 （Alpha版，不稳定！）
from google.colab import drive
drive.mount('/content/drive')
import os
# 你的模型路径和工作路径
model_dir = '/content/drive/MyDrive/RWKV_model'
os.makedirs(model_dir, exist_ok=True)
%cd $model_dir
!pip install torch transformers sentencepiece==0.1.97 pytorch-lightning==1.9.5 deepspeed==0.9.2 wandb ninja rwkv==0.7.4 lm_dataformat tqdm ftfy

Mounted at /content/drive
/content/drive/MyDrive/RWKV_model
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 100.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.3/779.3 kB 69.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.3/395.3 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 8.2 MB/s eta 0:00:

In [2]:
#@title 微调用模型下载
#@markdown 模型名称

base_model_name = "RWKV-4-World-0.4B-v1-20230529-ctx4096.pth" #@param {type:"string"}
base_model_url = f"https://huggingface.co/BlinkDL/rwkv-4-world/resolve/main/{base_model_name}"

In [3]:
!wget $base_model_url # 若已经下载请跳过

--2023-06-04 12:49:22--  https://huggingface.co/BlinkDL/rwkv-4-world/resolve/main/RWKV-4-World-0.4B-v1-20230529-ctx4096.pth
Resolving huggingface.co (huggingface.co)... 18.155.68.38, 18.155.68.116, 18.155.68.121, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/7f/9c/7f9cfb689f066fd31f71b817e907c34f0b02f23e488bf7de8c808191cba731f0/4b4a2733cf5e5dc97dd62106f391d99895d16b11c5ccd10c89f28c52067a4919?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27RWKV-4-World-0.4B-v1-20230529-ctx4096.pth%3B+filename%3D%22RWKV-4-World-0.4B-v1-20230529-ctx4096.pth%22%3B&Expires=1686142163&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzdmLzljLzdmOWNmYjY4OWYwNjZmZDMxZjcxYjgxN2U5MDdjMzRmMGIwMmYyM2U0ODhiZjdkZThjODA4MTkxY2JhNzMxZjAvNGI0YTI3MzNjZjVlNWRjOTdkZDYyMTA2ZjM5MWQ5OTg5NWQxNmIxMWM1Y2NkMTBjODlmMjhjNTIwNjdhNDkxOT9yZXNwb

In [ ]:
#@title 运行、微调、分词脚本下载（若已经下载可以更新）
#@markdown 这里设置为强制更新，请使用最新版本
force_clone = True #@param {type: 'boolean'}
if force_clone:
  !rm -rf RWKV-LM 
  !rm -rf json2binidx_tool
!git clone https://github.com/BlinkDL/RWKV-LM 
!git clone -b rwkv-tokenizer https://github.com/cahya-wirawan/json2binidx_tool

Cloning into 'RWKV-LM'...
remote: Enumerating objects: 1795, done.
remote: Counting objects: 100% (578/578), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 1795 (delta 540), reused 463 (delta 462), pack-reused 1217
Receiving objects: 100% (1795/1795), 11.16 MiB | 16.28 MiB/s, done.
Resolving deltas: 100% (1137/1137), done.
Cloning into 'json2binidx_tool'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 34 (delta 11), reused 18 (delta 8), pack-reused 0
Unpacking objects: 100% (34/34), 998.61 KiB | 2.20 MiB/s, done.


In [4]:
tokenized_name = "tokenized" #@param {type:"string"}

In [5]:
#@title 语料和分词准备 （此部分仅用CPU完成，不必连接GPU机器；如已经分词请跳过）
# 请自己准备语料，这里只提供示例
corpus_directory = '/content/drive/MyDrive/corpus'#@param {type:"string"}

In [ ]:
import os
def get_files(folder, suffix=".txt", subdirs=True): # 也可使用 .json 或 .jsonl 格式
  file_dirs = []
  if subdirs:
    for root, dirs, files in os.walk(folder):
      for file in files:
        if file.endswith(suffix):
          file_dir = os.path.join(root, file)
          file_dirs.append(file_dir)
  else:
    for file in os.listdir(folder):
      if file.endswith(suffix):
        file_dir = os.path.join(folder, file)
        file_dirs.append(file_dir)
  return file_dirs

In [ ]:
input_files = ','.join(get_files(corpus_directory))
command = f'python ./json2binidx_tool/tools/preprocess_data.py --input {input_files} --output-prefix {tokenized_name} --vocab ./json2binidx_tool/rwkv_vocab_v20230424.txt --dataset-impl mmap --tokenizer-type RWKVTokenizer --append-eod'
!{command}


#######################################################################################################################

This tokenizer is not used in any RWKV models yet. I plan to use it for the future multilang RWKV models.

Benefits:

* Good support of most languages, from European to CJK to Arabic and Hindi and more.

* Clean vocab. Good for code too. Vocab size = 65525 (use 0 for <|endoftext|>).

* Good at numbers: the numerical tokens are '0'~'9', '10'~'99', ' 0'~' 9', ' 10'~' 99'.

* Very easy tokenization:

** The input text must be in UTF-8.

** Greedy encoding: always pick the longest (in bytes) token (with the highest id) that matches your UTF-8 bytes.

* The tokenization result is surprisingly good, because the vocab respects word boundaries and UTF-8 boundaries.

For 10x faster speed:
mypyc rwkv_tokenizer.py
python3 -c "import rwkv_tokenizer"

#######################################################################################################################

> buildi

In [6]:
#@title 训练，必须连接GPU机器
ctxlen = 1024
n_layer = 24
n_embd = 1024
# 因为参数密集，不推荐半精度量化，但是谷歌云盘容量有限
precision = "bf16" 
%cd ./RWKV-LM/RWKV-v4neo
command = f'python "./train.py" --load_model "../../{base_model_name}" --proj_dir "../../finetuned" --data_file "../../{tokenized_name}_text_document" --data_type "binidx" --vocab_size 65536 --ctx_len {ctxlen} --epoch_steps 16 --epoch_count 200 --epoch_begin 0 --epoch_save 20 --micro_bsz 2 --n_layer {n_layer} --n_embd {n_embd} --pre_ffn 0 --head_qk 0 --lr_init 1e-4 --lr_final 5e-6 --warmup_steps 50 --beta1 0.9 --beta2 0.999 --adam_eps 1e-8 --accelerator gpu --devices 1 --precision {precision} --strategy deepspeed_stage_2 --grad_cp 0'
!{command}

/content/drive/MyDrive/RWKV_model/RWKV-LM/RWKV-v4neo
########## work in progress ##########

############################################################################
#
# RWKV-4 BF16 on 1x1 GPU, bsz 1x1x2=2, deepspeed_stage_2 
#
# Data = ../../tokenized_text_document (binidx), ProjDir = ../../finetuned
#
# Epoch = 0 to 199 (will continue afterwards), save every 20 epoch
#
# Each "epoch" = 16 steps, 32 samples, 32768 tokens
#
# Model = 24 n_layer, 1024 n_embd, 1024 ctx_len
#
# Adam = lr 0.0001 to 5e-06, warmup 50 steps, beta (0.9, 0.999), eps 1e-08
#
# Found torch 2.0.1+cu118, recommend 1.13.1+cu117 or newer
# Found deepspeed 0.9.2, recommend 0.7.0 (faster than newer versions)
# Found pytorch_lightning 1.9.5, recommend 1.9.1 or newer
#
############################################################################

{'load_model': '../../RWKV-4-World-0.4B-v1-20230529-ctx4096.pth', 'wandb': '', 'proj_dir': '../../finetuned', 'random_seed': -1, 'data_file': '../../tokenized_text_document',